# Q3

In [3]:
import numpy as np
from numpy.linalg import inv,matrix_rank,det
from sklearn.preprocessing import PolynomialFeatures

X = np.array([[1, 1.5],[0.9, 2], [3, 1]])
y = np.array([1, 2.6, 3])
order = 3
poly = PolynomialFeatures(order)
X_poly = poly.fit_transform(X)
# check if X^T * X is invertible
def check_inverse_rank(matrix, tol=1e-15):
    rank = matrix_rank(matrix)
    print("matrix rank is : "+ str(rank))
    print("matrix size is : "+ str(matrix.shape))

    if matrix.shape[0] == matrix.shape[1]:
       if rank == matrix.shape[0]:
           print("matrix is invertible")
       else:
           print("matrix is not invertible")
    else:
       print("matrix is not square, hence not invertible")

    return (rank == matrix.shape[0]) and (matrix.shape[0] == matrix.shape[1])
check_inverse_rank(X_poly.T @ X_poly)

matrix rank is : 3
matrix size is : (10, 10)
matrix is not invertible


False

# Q8

In [6]:
X = np.array([[3, 15, 50],[5, 12, 68], [2, 9, 60], [4, 19, 80]])
y = np.array([5,6,3,8])
lamda = 0.1
order = 1
poly = PolynomialFeatures(order)
X_poly = poly.fit_transform(X)
# check if X^T * X is invertible
if check_inverse_rank(X_poly.T @ X_poly):
    w = inv(X_poly.T @ X_poly) @ X_poly.T @ y
else:
    reg_L = lamda*np.identity(X_poly.shape[1])
    w = inv(X_poly.T @ X_poly + reg_L) @ X_poly.T @ y

# prediction
x_test = np.array([[7, 10, 70]])
x_test_poly = poly.fit_transform(x_test)
y_test = x_test_poly @ w
print(y_test)

matrix rank is : 4
matrix size is : (4, 4)
matrix is invertible
[6.66030534]


# Q9

In [ ]:
def multi_logistic_cost_gradient(X, W, Y, eps=1e-15):

    # Compute prediction, cost and gradient based on cross entropy
    z = X @ W
    exp_z = np.exp(z)
    pred_Y = exp_z / np.sum(exp_z, axis=-1, keepdims=True)
    cost   = np.sum(-(Y * np.log(pred_Y)))/X.shape[0]
    gradient = X.T @ (pred_Y-Y) / X.shape[0]

    return pred_Y, cost, gradient

In [14]:
from sklearn.preprocessing import OneHotEncoder
# Step 0: prepare data
X = np.array([[0.5, 1.2, -0.3],[-1, 0.8, 1.5],[2.3, -0.7, 0.5],[0, 1.5, -1]])
poly = PolynomialFeatures(1)
P = poly.fit_transform(X)

y_raw = np.array([[1], [2], [3], [1]])
onehot_encoder=OneHotEncoder(sparse_output=False)
Y = onehot_encoder.fit_transform(y_raw)

lr = 0.1
# Step 1: initialize weights
W = np.array([[0, 0, 0],[0.01, -0.02, 0.03],[0.05, 0.04, -0.01],[-0.03, 0.02, 0.01]])

# Step 2: perform one step of gradient descent
pred_Y, cost, gradient = multi_logistic_cost_gradient(P, W, Y, eps=1e-15)
W = W - lr*gradient
print("Updated weights after 1st iteration: \n", W)

Updated weights after 1st iteration: 
 [[ 0.01625831 -0.00835474 -0.00790358]
 [ 0.00756886 -0.05825139  0.07068253]
 [ 0.09301149  0.03591212 -0.04892361]
 [-0.06720425  0.05104296  0.01616129]]


# Q10

In [15]:
def ReLU(x):
    return np.maximum(0, x)
def Softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=-1, keepdims=True) 

In [19]:
X = np.array([[0.5, 1.2, -0.3],[-1, 0.8, 1.5],[2.3, -0.7, 0.5],[0, 1.5, -1]])
poly = PolynomialFeatures(1)
P = poly.fit_transform(X)

y_raw = np.array([[1], [2], [3], [1]])
onehot_encoder=OneHotEncoder(sparse_output=False)
Y = onehot_encoder.fit_transform(y_raw)

lr = 0.1
# Step 1: initialize weights
W1_0 = W2_0 = np.array([[0, 0, 0],[0.01, -0.02, 0.03],[0.05, 0.04, -0.01],[-0.03, 0.02, 0.01]])

### Forward Pass ###
# first layer output
L1=ReLU(P @ W1_0)
print("first layer output:", L1)

# second layer output
# Column of 1s (2x1)
ones = np.ones((L1.shape[0], 1))
# Concatenate along columns (axis=1)
L1_with_ones = np.hstack((ones, L1))
L2=Softmax(L1_with_ones @ W2_0)
print("second layer output:", L2)

### Backward Pass ###
# Error at Second layer
E2 = L2-Y
print("Error at Second layer:", E2)
# Gradient at Second layer
G2 = (L1_with_ones.T @ E2)/X.shape[0]
print("Gradient at Second layer:", G2)
# Weights update at Second layer
W2_1 = W2_0 - lr * G2
print("Weights update at Second layer:", W2_1)
# Error at First layer
E1 = E2 @ W2_0[1:].T * (L1 > 0) #(L1>0)=(Z1>0), which is the derivative of ReLU.
print("Error at First layer:", E1)
# Gradient at First layer
G1 = (P.T @ E1)/X.shape[0]
print("Gradient at First layer:", G1)
# Weights update at First layer
W1_1 = W1_0 - lr * G1
print("Weights update at First layer:", W1_1)

first layer output: [[0.074 0.032 0.   ]
 [0.    0.082 0.   ]
 [0.    0.    0.081]
 [0.105 0.04  0.   ]]
second layer output: [[0.3336644  0.33281797 0.33351762]
 [0.33397095 0.3336972  0.33233185]
 [0.33252381 0.33387326 0.33360293]
 [0.33376095 0.3325782  0.33366084]]
Error at Second layer: [[-0.6663356   0.33281797  0.33351762]
 [ 0.33397095 -0.6663028   0.33233185]
 [ 0.33252381  0.33387326 -0.66639707]
 [-0.66623905  0.3325782   0.33366084]]
Gradient at Second layer: [[-0.16651997  0.08324166  0.08327831]
 [-0.02981598  0.01488731  0.01492867]
 [-0.00514667 -0.00767088  0.01281755]
 [ 0.00673361  0.00676093 -0.01349454]]
Weights update at Second layer: [[ 0.016652   -0.00832417 -0.00832783]
 [ 0.0129816  -0.02148873  0.02850713]
 [ 0.05051467  0.04076709 -0.01128176]
 [-0.03067336  0.01932391  0.01134945]]
Error at First layer: [[-0.00331419 -0.02333924  0.        ]
 [ 0.         -0.01327688 -0.        ]
 [-0.          0.         -0.00996222]
 [-0.00330413 -0.02334543  0.        ]